In [1]:
!rm -rf antihydrogen_test.csv.gz
!rm -rf antihydrogen_test.csv
!rm -rf antihydrogen_train.csv.gz
!rm -rf antihydrogen_train.csv
!rm -rf antihydrogen_valid.csv.gz
!rm -rf antihydrogen_valid.csv

!wget http://mlphysics.ics.uci.edu/data/antihydrogen/antihydrogen_test.csv.gz
!wget http://mlphysics.ics.uci.edu/data/antihydrogen/antihydrogen_train.csv.gz
!wget http://mlphysics.ics.uci.edu/data/antihydrogen/antihydrogen_valid.csv.gz

!gunzip antihydrogen_test.csv.gz
!gunzip antihydrogen_train.csv.gz
!gunzip antihydrogen_valid.csv.gz

--2025-01-08 12:07:29--  http://mlphysics.ics.uci.edu/data/antihydrogen/antihydrogen_test.csv.gz
Resolving mlphysics.ics.uci.edu (mlphysics.ics.uci.edu)... 128.195.1.86
Connecting to mlphysics.ics.uci.edu (mlphysics.ics.uci.edu)|128.195.1.86|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6608724 (6.3M) [application/x-gzip]
Saving to: ‘antihydrogen_test.csv.gz’

antihydrogen_test.c 100%[===================>]   6.30M  14.4MB/s    in 0.4s    

2025-01-08 12:07:30 (14.4 MB/s) - ‘antihydrogen_test.csv.gz’ saved [6608724/6608724]

--2025-01-08 12:07:30--  http://mlphysics.ics.uci.edu/data/antihydrogen/antihydrogen_train.csv.gz
Resolving mlphysics.ics.uci.edu (mlphysics.ics.uci.edu)... 128.195.1.86
Connecting to mlphysics.ics.uci.edu (mlphysics.ics.uci.edu)|128.195.1.86|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19840043 (19M) [application/x-gzip]
Saving to: ‘antihydrogen_train.csv.gz’

antihydrogen_train. 100%[===================>]  18.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

train = pd.read_csv('antihydrogen_train.csv',sep=' ',dtype='int8')#,nrows=60000)
test = pd.read_csv('antihydrogen_test.csv',sep=' ', dtype = 'int8')#,nrows=20000)
valid = pd.read_csv('antihydrogen_valid.csv',sep=' ', dtype = 'int8')#,nrows=20000)

In [5]:
target_train = train['0']
target_test = test['1']
target_valid = valid['0']

x_train = train[train.columns[1:1431]]
x_test = test[test.columns[1:1431]]
x_valid = valid[valid.columns[1:1431]]

In [6]:

#definizione modello come classica CNN con tre blocchi convoluzionali e 3 layer densi contando quello di output
inputs = keras.Input(shape=(1430,1))

x = keras.layers.Reshape((55,26,1))(inputs)


x = keras.layers.Conv2D(32, kernel_size=(3,3))(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.ReLU()(x)
x = keras.layers.MaxPool2D((2,2))(x)

x = keras.layers.Conv2D(64, kernel_size=(3,3))(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.ReLU()(x)
x = keras.layers.MaxPool2D((2,2))(x)

x = keras.layers.Conv2D(64, kernel_size=(3,3))(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.ReLU()(x)

x = keras.layers.Flatten()(x)


x = keras.layers.Dense(256)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.ReLU()(x)
x = keras.layers.Dense(128)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.ReLU()(x)

outputs = keras.layers.Dense(1)(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()
keras.utils.plot_model(model, "conv_2D_model.png", show_shapes=True)


LR_ST=1e-3


OPTIMIZER = tf.keras.optimizers.SGD(learning_rate=LR_ST)

LOSS= tf.keras.losses.MeanSquaredError()

#METRIC = [tf.keras.metrics.CategoricalAccuracy()]
METRIC = ['accuracy']
#METRIC2 = ['mae']
model.compile(optimizer=OPTIMIZER,loss=LOSS,metrics=METRIC,)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 1430, 1)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_1 (Reshape)                  │ (None, 55, 26, 1)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 53, 24, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 53, 24, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu (ReLU)                         │ (None, 53, 24, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 26, 12, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 24, 10, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 24, 10, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_1 (ReLU)                       │ (None, 24, 10, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 12, 5, 64)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 10, 3, 64)           │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 10, 3, 64)           │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_2 (ReLU)                       │ (None, 10, 3, 64)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1920)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         491,776 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_3 (ReLU)                       │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │              

 Total params: 582,721 (2.22 MB)

 Trainable params: 581,633 (2.22 MB)

 Non-trainable params: 1,088 (4.25 KB)

In [7]:
history = model.fit(x_train, target_train, epochs=20, batch_size=128,validation_data=(x_valid,target_valid), shuffle=True, verbose=1)

Epoch 1/20
 650/7480 ━━━━━━━━━━━━━━━━━━━━ 50:27 443ms/step - accuracy: 0.5187 - loss: 0.5937

KeyboardInterrupt: 

In [ ]:
hist = history.history
x_arr = np.arange(len(hist['loss'])) + 1

fig = plt.figure(figsize=(12, 4))
ax = fig.add_subplot(1, 2, 1)
ax.plot(x_arr, hist['loss'], '-o', label='Train loss')
ax.plot(x_arr, hist['val_loss'], '--<', label='Validation loss')
ax.legend(fontsize=15)
ax.set_xlabel('Epoch', size=15)
ax.set_ylabel('Loss', size=15)

ax = fig.add_subplot(1, 2, 2)
ax.plot(x_arr, hist['accuracy'], '-o', label='Train acc.')
ax.plot(x_arr, hist['val_accuracy'], '--<', label='Validation acc.')
ax.legend(fontsize=15)
ax.set_xlabel('Epoch', size=15)
ax.set_ylabel('Accuracy', size=15)
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(x_test,  target_test, verbose=2)
print('\nTest loss:', test_loss)
print('\nTest accuracy:', test_acc)

In [ ]:
predictions = model.predict(x_test)
predictions = predictions.reshape((predictions.shape[0]))

average_abs_diff = 0.0
for i in range(predictions.shape[0]):
  print('Predicted vs True : %f VS %f Diff= %f' % (predictions[i],target_test[i],predictions[i]-target_test[i] ))
  average_abs_diff = average_abs_diff + abs(predictions[i]-target_test[i])

print('Sample MAE: ', average_abs_diff/predictions.shape[0])

In [ ]:
for i in range(0,len(predictions)):
  if predictions[i] <= 0.5:
    predictions[i]=0
  if predictions[i] > 0.5:
    predictions[i]=1

plt.figure(figsize=(15, 10))
plt.subplot(1, 3, 1)
plt.hist(target_test)
plt.title('Istogramma del target del test')

plt.subplot(1, 3, 2)
plt.hist(predictions)
plt.title('Istogramma dei valori predetti dal modello')

plt.subplot(1, 3, 3)
plt.hist(target_test,label='valori veri')
plt.hist(predictions,label='valori predetti')
plt.legend(loc='lower right')
plt.title('Istogrammi dei valori predetti vs valori veri')

In [ ]:
plt.figure(figsize=(15, 10))
plt.hist(np.abs(predictions-target_test),label='|valori predetti-valori veri|')
plt.title('Istogrammi dei valori predetti vs valori veri')